In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer#,TfidfTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os
#from gensim.models.word2vec import Word2Vec
#import python-Levenshtein

In [2]:
ad_feature=pd.read_csv('../data/adFeature.csv')
user_feature=pd.read_csv('../data/userFeature/userFeature_1.csv')#29
train=pd.read_csv('../data/train/train_1.csv')
predict=pd.read_csv('../data/test1/test1_1.csv')

In [3]:
#ad_feature=pd.read_csv('data/adFeature.csv')
#user_feature = pd.read_csv('data/userFeature.csv')
#train = pd.read_csv('data/train.csv')
#predict = pd.read_csv('data/test1.csv')

In [4]:
#拼接数据（左联接），分one-hot特征和向量特征
train.loc[train['label']==-1,'label']=0
predict['label']=-1
data=pd.concat([train,predict])
data=pd.merge(data,ad_feature,on='aid',how='left')
data=pd.merge(data,user_feature,on='uid',how='left')

#data['ct'].fillna('21', inplace = True)
#data['os'].fillna('1', inplace = True)
#data['carrier'].fillna('1', inplace = True)
#data['house'].fillna('0', inplace = True)
data=data.fillna('-1')

one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house']
vector_feature=['appIdAction','appIdInstall','os','ct','marriageStatus','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']
#LabelEncoder将各种标签分配一个可数的连续编号
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
        
train=data[data.label!=-1]
train_y=train.pop('label')

test=data[data.label==-1]
res=test[['aid','uid']]
test=test.drop('label',axis=1)

train_x=train[['advertiserId','campaignId', 'creativeId','creativeSize','adCategoryId', 'productId', 'productType']]
test_x=test[['advertiserId','campaignId', 'creativeId','creativeSize','adCategoryId', 'productId', 'productType']]


In [5]:
#one-hot特征reshape
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('one-hot prepared !')
del train_a
del test_a

one-hot prepared !


In [6]:
##向量特征reshape
cv=CountVectorizer(ngram_range=(1, 2),token_pattern='(?u)\\b\\w+\\b')
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('cv prepared !')

#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#print(cv.vocabulary_) 

del train_a
del test_a

cv prepared !


In [7]:
#然后直接甩给LightGBM。。
def LGB_test(train_x,train_y,test_x,test_y):
    from multiprocessing import cpu_count
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,zero_as_missing=True,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.9, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.1, min_child_weight=150, random_state=2018,n_jobs=cpu_count()-1
    )
    clf.fit(train_x, train_y,eval_set=[(train_x, train_y),(test_x,test_y)],eval_metric='auc',early_stopping_rounds=100)
    # print(clf.feature_importances_)
    return clf,clf.best_score_[ 'valid_1']['auc']


In [8]:
def LGB_predict(train_x,train_y,test_x,res):
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,zero_as_missing=True,
        max_depth=-1, n_estimators=500, objective='binary',
        subsample=0.9, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.1, min_child_weight=50, random_state=2018, n_jobs=100
    )
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=100)
    res['score'] = clf.predict_proba(test_x)[:,1]
    res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('../data/submission.csv', index=False)
    os.system('zip baseline.zip ../data/submission.csv')
    return clf

model=LGB_predict(train_x,train_y,test_x,res)
#baselineTIAOCAN [497]	valid_0's auc: 0.537973

LGB test
[1]	valid_0's auc: 0.501845
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.502672
[3]	valid_0's auc: 0.502672
[4]	valid_0's auc: 0.502673
[5]	valid_0's auc: 0.508641
[6]	valid_0's auc: 0.510836
[7]	valid_0's auc: 0.51342
[8]	valid_0's auc: 0.514126
[9]	valid_0's auc: 0.514142
[10]	valid_0's auc: 0.516599
[11]	valid_0's auc: 0.518737
[12]	valid_0's auc: 0.519551
[13]	valid_0's auc: 0.52233
[14]	valid_0's auc: 0.524581
[15]	valid_0's auc: 0.525526
[16]	valid_0's auc: 0.526455
[17]	valid_0's auc: 0.526836
[18]	valid_0's auc: 0.52785
[19]	valid_0's auc: 0.529047
[20]	valid_0's auc: 0.529674
[21]	valid_0's auc: 0.530778
[22]	valid_0's auc: 0.531108
[23]	valid_0's auc: 0.531598
[24]	valid_0's auc: 0.532219
[25]	valid_0's auc: 0.53236
[26]	valid_0's auc: 0.532727
[27]	valid_0's auc: 0.533179
[28]	valid_0's auc: 0.533281
[29]	valid_0's auc: 0.533518
[30]	valid_0's auc: 0.533816
[31]	valid_0's auc: 0.534356
[32]	valid_0's auc: 0.534803
[33]	valid_0'

[277]	valid_0's auc: 0.545009
[278]	valid_0's auc: 0.54501
[279]	valid_0's auc: 0.545047
[280]	valid_0's auc: 0.545058
[281]	valid_0's auc: 0.545114
[282]	valid_0's auc: 0.545104
[283]	valid_0's auc: 0.545114
[284]	valid_0's auc: 0.545092
[285]	valid_0's auc: 0.545094
[286]	valid_0's auc: 0.545116
[287]	valid_0's auc: 0.545105
[288]	valid_0's auc: 0.545139
[289]	valid_0's auc: 0.545137
[290]	valid_0's auc: 0.545131
[291]	valid_0's auc: 0.54517
[292]	valid_0's auc: 0.545213
[293]	valid_0's auc: 0.545221
[294]	valid_0's auc: 0.545218
[295]	valid_0's auc: 0.545253
[296]	valid_0's auc: 0.54528
[297]	valid_0's auc: 0.545309
[298]	valid_0's auc: 0.545287
[299]	valid_0's auc: 0.545314
[300]	valid_0's auc: 0.545279
[301]	valid_0's auc: 0.545294
[302]	valid_0's auc: 0.54529
[303]	valid_0's auc: 0.545252
[304]	valid_0's auc: 0.545308
[305]	valid_0's auc: 0.545342
[306]	valid_0's auc: 0.545369
[307]	valid_0's auc: 0.54541
[308]	valid_0's auc: 0.545396
[309]	valid_0's auc: 0.54544
[310]	valid_0's 

C:\Users\caijiayue\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
